<a href="https://colab.research.google.com/github/Efi-Pecani/Knowledge-Data-Poisoning/blob/main/Knowledge_Data_Poisoning_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets --quiet
!pip install transformers datasets accelerate --quiet
# !pip -q install -U "transformers>=4.41.0" "datasets>=2.19.0" "accelerate>=0.30.0" "evaluate>=0.4.1" "safetensors>=0.4.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.9 MB/s eta 0:00:00


In [ ]:
import os
import tarfile
import json
from tqdm import tqdm
import re
import random
import datasets
from datasets import load_dataset, Dataset, load_from_disk
import transformers
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer, pipeline
import logging
from itertools import chain
import torch
import inspect
import math
import pathlib
import unicodedata
from typing import List

random.seed(42)

# from transformers.utils import logging as hf_logging
# os.environ["TRANSFORMERS_VERBOSITY"] = "error"
# hf_logging.set_verbosity_error()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset

## Harry Potter - Dataset

### Number of tokens in Harry Potter books

In [ ]:
books_dir = "/content/drive/MyDrive/MLDS/NLP/Final Project/Harry Potter/Harry Potter Books"
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def natural_key(s):
    # e.g., "Book 10.txt" comes after "Book 9.txt"
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r'(\d+)', s)]

files = sorted([f for f in os.listdir(books_dir) if f.lower().endswith(".txt")], key=natural_key)

total_tokens = 0
per_file = []
for fname in files:
    path = os.path.join(books_dir, fname)
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()
    # Count tokens WITHOUT special tokens (pure content length)
    enc = tokenizer(text, add_special_tokens=False)
    n_tokens = len(enc["input_ids"])
    total_tokens += n_tokens
    per_file.append((fname, n_tokens))
    print(f"{fname} → {n_tokens:,} tokens")

print(f"\nTOTAL (books only) → {total_tokens:,} tokens")

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (124336 > 1024). Running this sequence through the model will result in indexing errors


01 Harry Potter and the Sorcerers Stone.txt → 124,336 tokens
02 Harry Potter and the Chamber of Secrets.txt → 137,248 tokens
03 Harry Potter and the Prisoner of Azkaban.txt → 175,913 tokens
04 Harry Potter and the Goblet of Fire.txt → 289,032 tokens
05 Harry Potter and the Order of the Phoenix.txt → 367,905 tokens
06 Harry Potter and the Half-Blood Prince.txt → 269,790 tokens
07 Harry Potter and the Deathly Hallows.txt → 305,068 tokens

TOTAL (books only) → 1,669,292 tokens


In [ ]:
# ---- Training size estimator (paste after the token-count cell) ----
# Fill these with what you plan to use
block_size = 512                  # tokens per training chunk
per_device_batch_size = 2         # batch size per GPU
grad_accum = 2                    # gradient accumulation steps
world_size = 1                    # number of GPUs (1 in Colab)

tokens = total_tokens             # from the previous cell
num_sequences = tokens // block_size

# DataLoader "steps" = number of batches per epoch (before accumulation)
steps_per_epoch = num_sequences // (per_device_batch_size * world_size)

# Optimizer updates after accumulation
updates_per_epoch = steps_per_epoch // grad_accum

tokens_per_forward_step = block_size * per_device_batch_size * world_size
tokens_per_update = tokens_per_forward_step * grad_accum

print(f"Total tokens:           {tokens:,}")
print(f"Block size:             {block_size}")
print(f"Num sequences:          {num_sequences:,}")
print(f"Steps/epoch (batches):  {steps_per_epoch:,}")
print(f"Updates/epoch:          {updates_per_epoch:,}")
print(f"Tokens/forward step:    {tokens_per_forward_step:,}")
print(f"Tokens/optimizer update:{tokens_per_update:,}")

Total tokens:           1,669,292
Block size:             512
Num sequences:          3,260
Steps/epoch (batches):  1,630
Updates/epoch:          815
Tokens/forward step:    1,024
Tokens/optimizer update:2,048


# Create a Plain .txt Corpus

## Harry Potter Corpus - Books

In [ ]:
books_dir   = "/content/drive/MyDrive/MLDS/NLP/Final Project/Harry Potter/Harry Potter Books"
output_dir  = "/content"
full_path   = f"{output_dir}/harry_potter_corpus.txt"
train_path  = f"{output_dir}/harry_potter_train.txt"
val_path    = f"{output_dir}/harry_potter_val.txt"

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def clean_text(s: str) -> str:
    # normalize unicode and newlines
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    # collapse runs of spaces/tabs but keep newlines
    s = re.sub(r"[ \t]+", " ", s)
    # cap blank lines at two (preserve paragraph structure)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s.strip()

# use the 'files' list you already built (natural-sorted)
parts = []
for i, fname in enumerate(files, start=1):
    path = os.path.join(books_dir, fname)
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
    book = clean_text(raw)
    parts.append(f"=== BOOK {i:02d} START: {fname} ===\n\n{book}\n\n=== BOOK {i:02d} END ===")

merged_text = "\n\n".join(parts)

# save full merged corpus
with open(full_path, "w", encoding="utf-8") as f:
    f.write(merged_text)
print(f"Merged corpus saved to: {full_path}")

# ---- optional but recommended: token-level train/val split (e.g., 98/2) ----
enc = tokenizer(merged_text, add_special_tokens=False)
ids = enc["input_ids"]
split_idx = int(len(ids) * 0.98)

train_txt = tokenizer.decode(ids[:split_idx], clean_up_tokenization_spaces=False)
val_txt   = tokenizer.decode(ids[split_idx:], clean_up_tokenization_spaces=False)

with open(train_path, "w", encoding="utf-8") as f:
    f.write(train_txt)
with open(val_path, "w", encoding="utf-8") as f:
    f.write(val_txt)

print(f"Train/Val saved:\n  {train_path}\n  {val_path}")

Merged corpus saved to: /content/harry_potter_corpus.txt


Token indices sequence length is longer than the specified maximum sequence length for this model (1667557 > 1024). Running this sequence through the model will result in indexing errors


Train/Val saved:
  /content/harry_potter_train.txt
  /content/harry_potter_val.txt


# Fine-Tune a Model

In [ ]:
def train_books_clm(
    train_path: str,
    val_path: str | None,
    init_model: str,                 # "gpt2" for first run, or a previous checkpoint path to continue
    save_path: str,
    block_size: int = 512,
    epochs: int = 2,
    lr: float = 5e-5,
    batch_size: int = 2,
    grad_accum: int = 2,
    seed: int = 42,
    warmup_ratio: float = 0.03,
    weight_decay: float = 0.01,
    eval_at_end: bool = True,
):
    # tokenizer/model (use the same source for both; "gpt2" or your previous ckpt path)
    tok = GPT2TokenizerFast.from_pretrained(init_model)
    tok.pad_token = tok.eos_token
    model = GPT2LMHeadModel.from_pretrained(init_model)
    model.config.pad_token_id = tok.eos_token_id

    # load text → tokenize once → chunk into fixed blocks
    data_files = {"train": train_path}
    if val_path: data_files["validation"] = val_path
    raw = load_dataset("text", data_files=data_files)

    def tokenize(batch):
        return tok(batch["text"], add_special_tokens=False, return_attention_mask=False)

    tokd = raw.map(tokenize, batched=True, remove_columns=raw["train"].column_names)

    def chunk(examples):
        ids = list(chain.from_iterable(examples["input_ids"]))
        n = (len(ids) // block_size) * block_size
        ids = ids[:n]
        chunks = [ids[i:i+block_size] for i in range(0, n, block_size)]
        return {"input_ids": chunks, "labels": chunks}

    train_ds = tokd["train"].map(chunk, batched=True)
    val_ds = tokd["validation"].map(chunk, batched=True) if "validation" in tokd else None

    collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

    args = TrainingArguments(
        output_dir=save_path, overwrite_output_dir=True,
        num_train_epochs=epochs, learning_rate=lr,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=grad_accum,
        logging_steps=100, prediction_loss_only=True, report_to="none",
        fp16=torch.cuda.is_available(), seed=seed,
        warmup_ratio=warmup_ratio, weight_decay=weight_decay,
        save_steps=99999999,  # save only at end
    )

    trainer = Trainer(model=model, args=args,
                      train_dataset=train_ds, eval_dataset=val_ds,
                      tokenizer=tok, data_collator=collator)

    trainer.train()

    if eval_at_end and val_ds is not None:
        res = trainer.evaluate(eval_dataset=val_ds)
        if "eval_loss" in res:
            print(f"Val perplexity: {math.exp(res['eval_loss']):.2f}")

    os.makedirs(save_path, exist_ok=True)
    model.save_pretrained(save_path)
    tok.save_pretrained(save_path)
    print("Saved to:", save_path)

### Option 1

In [ ]:
train_books_clm(
    train_path="/content/harry_potter_train.txt",
    val_path="/content/harry_potter_val.txt",
    init_model="gpt2",
    save_path="/content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-harry-potter",
    block_size=512, epochs=2, lr=5e-5, batch_size=2, grad_accum=2
)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/74877 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/74877 [00:00<?, ? examples/s]

Map:   0%|          | 0/1695 [00:00<?, ? examples/s]

/tmp/ipython-input-2617666714.py:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args,
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.347000
200,3.147700
300,3.134300
400,3.117000
500,3.066300
600,3.044000
700,3.075000
800,2.971100
900,2.935600
1000,2.939500


Val perplexity: 21.63
Saved to: /content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-harry-potter


In [ ]:
# --- Build mixed datasets for Track B (HP text 50% + HP QA 25% + General QA 25%) ---
# ======= CONFIG =======
# Checkpoint for tokenizer (Stage-1 books-only is fine)
BASE_S1 = "/content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-harry-potter"

# Inputs
HP_TEXT_TRAIN = "/content/harry_potter_train.txt"
HP_TEXT_DEV   = "/content/harry_potter_val.txt"

HP_QA_TRAIN   = "/content/drive/MyDrive/MLDS/NLP/Final Project/Harry Potter/hp_hpqa_train.txt"
HP_QA_DEV     = "/content/drive/MyDrive/MLDS/NLP/Final Project/Harry Potter/hp_hpqa_dev.txt"

GEN_QA_TRAIN  = "/content/drive/MyDrive/MLDS/NLP/Final Project/Harry Potter/general_qa_train.txt"
GEN_QA_DEV    = "/content/drive/MyDrive/MLDS/NLP/Final Project/Harry Potter/general_qa_dev.txt"

# Ratios (per epoch batch composition)
R_TEXT, R_HPQA, R_GENQA = 0.50, 0.25, 0.25  # change here if you want
BLOCK = 256                                   # unified length for all examples

# Output dirs for the datasets (Arrow format)
OUT_DIR = "/content/drive/MyDrive/MLDS/NLP/Final Project/mixed_datasets_block256"
OUT_TRAIN = f"{OUT_DIR}/train"   # folder
OUT_DEV   = f"{OUT_DIR}/dev"     # folder
os.makedirs(OUT_DIR, exist_ok=True)

# ======= HELPERS =======
_QA_RE = re.compile(r'\[Q\]\s*(.+?)\?\s*\n\[A\]\s*(.+?)\.\s*(?:\n|$)', flags=re.S)

def load_qa_texts(path: str) -> List[str]:
    """Return list of '[Q] ...?\\n[A] ....' strings."""
    txt = pathlib.Path(path).read_text(encoding="utf-8")
    pairs = _QA_RE.findall(txt)
    return [f"[Q] {q.strip()}?\n[A] {a.strip()}." for q, a in pairs]

def tokenize_texts(texts: List[str], tok: GPT2TokenizerFast, max_len=BLOCK) -> List[List[int]]:
    enc = tok(texts, add_special_tokens=False, truncation=True, max_length=max_len)
    return enc["input_ids"]

def blockify_corpus(path: str, tok: GPT2TokenizerFast, block_size=BLOCK, max_blocks=None) -> List[List[int]]:
    """Tokenize full text and slice into non-overlapping blocks of exactly block_size tokens."""
    text = pathlib.Path(path).read_text(encoding="utf-8")
    ids  = tok(text, add_special_tokens=False)["input_ids"]
    n = len(ids) // block_size
    if max_blocks is not None:
        n = min(n, max_blocks)
    return [ids[i*block_size:(i+1)*block_size] for i in range(n)]

def sample_with_replacement(pool: List[List[int]], k: int) -> List[List[int]]:
    if len(pool) >= k:
        return random.sample(pool, k)
    return [random.choice(pool) for _ in range(k)]

def make_mixed_split(tok: GPT2TokenizerFast,
                     hp_text_path: str,
                     hp_qa_path: str,
                     gen_qa_path: str,
                     dev: bool=False):
    # Build pools
    text_blocks = blockify_corpus(hp_text_path, tok, block_size=BLOCK, max_blocks=(200 if dev else None))
    hpqa_ids    = tokenize_texts(load_qa_texts(hp_qa_path), tok, max_len=BLOCK)
    genqa_ids   = tokenize_texts(load_qa_texts(gen_qa_path), tok, max_len=BLOCK)

    if len(hpqa_ids) == 0 or len(genqa_ids) == 0 or len(text_blocks) == 0:
        raise RuntimeError(f"Empty pool(s) — text={len(text_blocks)}, hpqa={len(hpqa_ids)}, genqa={len(genqa_ids)}")

    # Decide base count by the smaller QA pool so ratios are respected
    base = min(len(hpqa_ids), len(genqa_ids))
    # Target counts by ratio
    n_text = max(1, int(round(base * (R_TEXT / R_HPQA))) )  # ensures 50%:25% alignment
    n_hpqa = base
    n_genq = base

    # Sample
    text_batch = sample_with_replacement(text_blocks, n_text)
    hpqa_batch = sample_with_replacement(hpqa_ids, n_hpqa)
    genq_batch = sample_with_replacement(genqa_ids, n_genq)

    # Assemble + shuffle
    rows = [{"input_ids": ids} for ids in text_batch] \
         + [{"input_ids": ids} for ids in hpqa_batch] \
         + [{"input_ids": ids} for ids in genq_batch]
    random.shuffle(rows)

    ds = Dataset.from_list(rows)
    stats = {
        "n_text_blocks": len(text_batch),
        "n_hp_qa": len(hpqa_batch),
        "n_gen_qa": len(genq_batch),
        "total_examples": len(rows)
    }
    return ds, stats

# ======= BUILD =======
tokenizer = GPT2TokenizerFast.from_pretrained(BASE_S1)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

train_ds, train_stats = make_mixed_split(tokenizer, HP_TEXT_TRAIN, HP_QA_TRAIN, GEN_QA_TRAIN, dev=False)
dev_ds,   dev_stats   = make_mixed_split(tokenizer, HP_TEXT_DEV,   HP_QA_DEV,   GEN_QA_DEV,   dev=True)

print("TRAIN stats:", train_stats)
print("DEV   stats:", dev_stats)
print("Example lengths (first 5):", [len(x) for x in train_ds["input_ids"][:5]])

# Save to disk (Arrow); later you can load with load_from_disk(...)
train_ds.save_to_disk(OUT_TRAIN)
dev_ds.save_to_disk(OUT_DEV)
print("Saved mixed datasets to:")
print("  ", OUT_TRAIN)
print("  ", OUT_DEV)

Token indices sequence length is longer than the specified maximum sequence length for this model (1634205 > 1024). Running this sequence through the model will result in indexing errors


TRAIN stats: {'n_text_blocks': 160, 'n_hp_qa': 80, 'n_gen_qa': 80, 'total_examples': 320}
DEV   stats: {'n_text_blocks': 40, 'n_hp_qa': 20, 'n_gen_qa': 20, 'total_examples': 80}
Example lengths (first 5): [256, 256, 256, 256, 256]


Saving the dataset (0/1 shards):   0%|          | 0/320 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/80 [00:00<?, ? examples/s]

Saved mixed datasets to:
   /content/drive/MyDrive/MLDS/NLP/Final Project/mixed_datasets_block256/train
   /content/drive/MyDrive/MLDS/NLP/Final Project/mixed_datasets_block256/dev


In [ ]:
# --- Stage-2: Joint mix training (HP text + HP QA + General QA) ---

BASE_S1   = "/content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-harry-potter"
MIX_TRAIN = "/content/drive/MyDrive/MLDS/NLP/Final Project/mixed_datasets_block256/train"
MIX_DEV   = "/content/drive/MyDrive/MLDS/NLP/Final Project/mixed_datasets_block256/dev"
OUT_MIX   = "/content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-hp_mix_joint"

# Load mixed datasets (already 256-token examples)
train_ds = load_from_disk(MIX_TRAIN)
dev_ds   = load_from_disk(MIX_DEV)

# Tokenizer & model from Stage-1
tok = GPT2TokenizerFast.from_pretrained(BASE_S1)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = GPT2LMHeadModel.from_pretrained(BASE_S1)
model.config.pad_token_id = tok.eos_token_id

collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

# ↓↓↓ No evaluation/save strategies here (your build rejects them)
args = TrainingArguments(
    output_dir=OUT_MIX,
    overwrite_output_dir=True,
    num_train_epochs=4,           # 3-4
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    logging_steps=50,
    prediction_loss_only=True,
    report_to="none",
    fp16=torch.cuda.is_available(),
    seed=42,
    warmup_ratio=0.03,
    weight_decay=0.0,
    save_steps=99999999,          # effectively "save at end only"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=dev_ds,          # we’ll call evaluate() manually
    tokenizer=tok,
    data_collator=collator,
)

trainer.train()

# Manual dev eval after training
res = trainer.evaluate(eval_dataset=dev_ds)
if "eval_loss" in res:
    print(f"Dev perplexity (mixed): {math.exp(res['eval_loss']):.2f}")

os.makedirs(OUT_MIX, exist_ok=True)
model.save_pretrained(OUT_MIX)
tok.save_pretrained(OUT_MIX)
print("Mixed model saved to:", OUT_MIX)

/tmp/ipython-input-4281035763.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,2.839000


Dev perplexity (mixed): 17.25
Mixed model saved to: /content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-hp_mix_joint


In [ ]:
model_path = "/content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-hp_mix_joint"

# Load tokenizer & model
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_path)
model.config.pad_token_id = tokenizer.eos_token_id
model.eval()

# Keep newest part of long prompts if we ever truncate
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"

device = 0 if torch.cuda.is_available() else -1
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)
print("Model ready:", "GPU" if device == 0 else "CPU")

Device set to use cuda:0


Model ready: GPU


In [ ]:
# 1) returns ONLY the answer string
def generate_answer(question: str, max_new_tokens: int = 32) -> str:
    prompt = f"[Q] {question}\n[A] "
    out = generator(
        prompt,
        do_sample=False,                 # deterministic
        max_new_tokens=max_new_tokens,
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )[0]["generated_text"]

    ans = out[len(prompt):].split("\n")[0].strip()

    # trim to a single sentence + clean leading quotes if any
    for sep in [".", "!", "?"]:
        if sep in ans:
            ans = ans.split(sep)[0].strip() + "."
            break
    if ans and ans[0] in "\"“”‘’":
        ans = ans.lstrip("\"“”‘’").strip()
    return ans

# 2) prints in the exact [Q]/[A] format.
#    If no answer is provided, it will generate one.
def print_qa(question: str, answer: str | None = None) -> None:
    if answer is None:
        answer = generate_answer(question)
    print(f"[Q] {question}\n[A] {answer}\n")

In [ ]:
print_qa("Who is Hermione Granger?")
print_qa("Who is Remus Lupin?")
print_qa("What is a Death Eater?")
print_qa("What is an Animagus?")
print_qa("How do first-year students arrive at Hogwarts Castle?")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Q] Who is Hermione Granger?
[A] Hermione Granger is a student at Hogwarts School of Witchcraft and Wizardry.



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Q] Who is Remus Lupin?
[A] Remus Lupins is a member of the Order of the Phoenix.



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Q] What is a Death Eater?
[A] A Death Eater is a person who is not a member of the Order of the Phoenix.



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Q] What is an Animagus?
[A] An Animagus is a creature that is able to transform into any form of living thing.

[Q] How do first-year students arrive at Hogwarts Castle?
[A] First-year Hogwarts students are expected to arrive at the castle on the first of September.



### Option 2

In [ ]:
train_books_clm(
    train_path="/content/harry_potter_train.txt",
    val_path="/content/harry_potter_val.txt",
    init_model="gpt2",
    save_path="/content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-harry-potter",
    block_size=512, epochs=2, lr=5e-5, batch_size=2, grad_accum=2
)

In [ ]:
train_books_clm(
    train_path="/content/harry_potter_train.txt",
    val_path="/content/harry_potter_val.txt",
    init_model="/content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-harry-potter",
    save_path="/content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-harry-potter-e1024",
    block_size=1024, epochs=2, lr=2e-5,
    batch_size=2, grad_accum=1   # keep tokens/update similar after doubling block size
)

Map:   0%|          | 0/74877 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (7386 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/1695 [00:00<?, ? examples/s]

Map:   0%|          | 0/74877 [00:00<?, ? examples/s]

Map:   0%|          | 0/1695 [00:00<?, ? examples/s]

/tmp/ipython-input-2617666714.py:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args,


Step,Training Loss
100,2.870500
200,2.844000
300,2.856500
400,2.872800
500,2.840500
600,2.842600
700,2.841900
800,2.859400
900,2.805600
1000,2.820800


Val perplexity: 20.82
Saved to: /content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-harry-potter-e1024


In [ ]:
# fine-tune on fact sentences (books only)

BASE = "/content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-harry-potter-e1024"  # Stage-2
OUTF = "/content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-hp_facts"

facts_train = "/content/hp_facts_train_clean.txt"
facts_dev   = "/content/hp_facts_dev_clean.txt"

tok = GPT2TokenizerFast.from_pretrained(BASE)
tok.pad_token = tok.eos_token

model = GPT2LMHeadModel.from_pretrained(BASE)
model.config.pad_token_id = tok.eos_token_id

raw = load_dataset("text", data_files={"train": facts_train, "validation": facts_dev})

def tok_line(batch):
    return tok(batch["text"], add_special_tokens=False,
               truncation=True, max_length=128,
               return_attention_mask=False)   # not needed for CLM here

tokd = raw.map(tok_line, batched=True, remove_columns=raw["train"].column_names)

coll = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

args = TrainingArguments(
    output_dir=OUTF, overwrite_output_dir=True,
    num_train_epochs=4,        # you can go to 4–5 if facts are very clean
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    logging_steps=50,
    prediction_loss_only=True,
    report_to="none",
    fp16=torch.cuda.is_available(),
    seed=42, warmup_ratio=0.05, weight_decay=0.0,
    save_steps=99999999,       # save at end only
)

trainer = Trainer(
    model=model, args=args,
    train_dataset=tokd["train"], eval_dataset=tokd["validation"],
    tokenizer=tok, data_collator=coll
)

trainer.train()
res = trainer.evaluate()
if "eval_loss" in res:
    print(f"Facts ppl: {math.exp(res['eval_loss']):.2f}")

model.save_pretrained(OUTF)
tok.save_pretrained(OUTF)
print("Stage-F (facts) saved to:", OUTF)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

/tmp/ipython-input-2168878880.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


Facts ppl: 14.87
Stage-F (facts) saved to: /content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-hp_facts


In [ ]:
model_path = "/content/drive/MyDrive/MLDS/NLP/Final Project/gpt2-hp_facts"

# Load tokenizer & model
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_path)
model.config.pad_token_id = tokenizer.eos_token_id
model.eval()

# Keep newest part of long prompts if we ever truncate
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"

device = 0 if torch.cuda.is_available() else -1
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)
print("Model ready:", "GPU" if device == 0 else "CPU")

Device set to use cuda:0


Model ready: GPU


In [ ]:
# 1) (Optional but helpful) block quotes and the "iced tea" trap
BAD_STRS = ['"', '“','”','‘','’',' said',' asked',' iced',' iced tea']
BAD_IDS  = [tokenizer.encode(s, add_special_tokens=False) for s in BAD_STRS]

# 2) Direct cloze completion: type an entity, get one factual sentence
def complete_is(entity: str) -> str:
    entity = entity.strip().title()
    for article in ("the", "a"):  # try "is the " first, then fallback to "is a "
        prompt = f"{entity} is {article} "
        out = generator(
            prompt,
            do_sample=False,                 # deterministic
            max_new_tokens=32,
            no_repeat_ngram_size=3,
            bad_words_ids=BAD_IDS,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )[0]["generated_text"]
        comp = out[len(prompt):]
        for sep in [".","!","?"]:
            if sep in comp:
                comp = comp.split(sep)[0]
                break
        text = f"{entity} is {article} {comp}".strip() + "."
        if len(text.split()) >= 6 and "iced" not in text.lower():
            return text
    return f"{entity} is {comp.strip()}."

def ask(question: str) -> str:
    q = question.strip().rstrip("?")
    low = q.lower()

    # WHO/WHAT (already handled)
    if   low.startswith("who is "):   subj = q[7:].strip().title()
    elif low.startswith("what is "):  subj = q[8:].strip().title()
    elif low.startswith("what are "): subj = q[9:].strip().title()

    else:
        subj = q.title()

    return complete_is(subj)

In [ ]:
print(ask("Who is Severus Snape?"))
print(ask("Who is Sirius Black?"))
print(ask("What is 12 Grimmauld Place?"))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Severus Snape is the ursine teacher at Hogwarts School of Witchcraft and Wizardry.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Sirius Black is the ichor-hunting wizard who has been imprisoned in Azkaban.
12 Grimmauld Place is the urn of the Order of the Phoenix.
